## Comprobar HW y SW instalado








In [2]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.00GHz
    CPU family:           6
    Model:                85
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             3
    BogoMIPS:             4000.26
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 cl
                          flush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc re
                          p_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3
                           fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand
                           hypervisor lahf_lm abm 3dnowprefetch i

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [4]:
!nvidia-smi

Tue Oct 15 15:31:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Tutorial: Multiplicación de matrices en GPU



Adaptación de los ejemplos disponibles en los siguientes enlaces:
*  https://medium.com/@harsh20111997/cuda-programming-2d-matrix-multiplication-1aa774b8d703
* https://www.quantstart.com/articles/Matrix-Matrix-Multiplication-on-the-GPU-with-Nvidia-CUDA/

In [5]:
! mkdir cuda

In [6]:
%%writefile cuda/MatMult.cu

#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>

#define N 3

__global__ void matrix_mul(int *a, int *b, int *c, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;
    int sum = 0;
    if (i < n && j < n) {
        for (int k = 0; k < n; k++)
            sum += a[i * n + k] * b[k * n + j];
        c[i * n + j] = sum;
    }
}

void printMat( int *mm, char *nombre, int n){
   for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++)
					printf(" %c[%d][%d] = %d ",nombre[0] ,i, j, mm[i * n + j]);
        printf("\n");
    }
}

int main() {
    int n = N;
    int *a, *b, *c;
    int *d_a, *d_b, *d_c;
    int size = n * n * sizeof(int);
		char matA[]= "A";
		char matB[]= "B";
		char matC[]= "C";

    a = (int *)malloc(size);
    b = (int *)malloc(size);
    c = (int *)malloc(size);

		// Inicializacion de valores
    for (int i = 0; i < n; i++)
        for (int j = 0; j < n; j++) {
            a[i * n + j] = i + j;
            b[i * n + j] = i * j;
        }

    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);

    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    dim3 blockSize(N, N);
    dim3 gridSize((n + N - 1) / N, (n + N - 1) / N);
    matrix_mul<<<gridSize, blockSize>>>(d_a, d_b, d_c, n);

    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    printMat( a, matA, n); printf("\n");
    printMat( b, matB, n); printf("\n");
		printMat( c, matC, n);

		/* for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++)
					printf(" c[ %d ][ %d ] = %d ", i, j, c[i * n + j]);
        printf("\n");
    }
		*/

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(a);
    free(b);
    free(c);

    return 0;
}

Writing cuda/MatMult.cu


In [7]:
!nvcc cuda/MatMult.cu -o cuda/MatMult
!ls -la cuda

total 964
drwxr-xr-x 2 root root   4096 Oct 15 15:31 .
drwxr-xr-x 1 root root   4096 Oct 15 15:31 ..
-rwxr-xr-x 1 root root 971280 Oct 15 15:31 MatMult
-rw-r--r-- 1 root root   1852 Oct 15 15:31 MatMult.cu


In [8]:
!./cuda/MatMult

 A[0][0] = 0  A[0][1] = 1  A[0][2] = 2 
 A[1][0] = 1  A[1][1] = 2  A[1][2] = 3 
 A[2][0] = 2  A[2][1] = 3  A[2][2] = 4 

 B[0][0] = 0  B[0][1] = 0  B[0][2] = 0 
 B[1][0] = 0  B[1][1] = 1  B[1][2] = 2 
 B[2][0] = 0  B[2][1] = 2  B[2][2] = 4 

 C[0][0] = 0  C[0][1] = 5  C[0][2] = 10 
 C[1][0] = 0  C[1][1] = 8  C[1][2] = 16 
 C[2][0] = 0  C[2][1] = 11  C[2][2] = 22 


### Ejercicio T1:
En el ejercicio anterior se utiliza un grid de bloques 2D y en cada bloque una distribución de hilos también 2D.
* Realice una version 1D tanto para el lanzamiento de bloques, como para la distribución de hilos dentro del bloque.
* ¿Que versión es más eficiente? Justifique los resultados.


In [9]:
!ls -la cuda/*

-rwxr-xr-x 1 root root 971280 Oct 15 15:31 cuda/MatMult
-rw-r--r-- 1 root root   1852 Oct 15 15:31 cuda/MatMult.cu


In [10]:
!nvprof cuda/MatMult

==754== NVPROF is profiling process 754, command: cuda/MatMult
 A[0][0] = 0  A[0][1] = 1  A[0][2] = 2 
 A[1][0] = 1  A[1][1] = 2  A[1][2] = 3 
 A[2][0] = 2  A[2][1] = 3  A[2][2] = 4 

 B[0][0] = 0  B[0][1] = 0  B[0][2] = 0 
 B[1][0] = 0  B[1][1] = 1  B[1][2] = 2 
 B[2][0] = 0  B[2][1] = 2  B[2][2] = 4 

 C[0][0] = 0  C[0][1] = 5  C[0][2] = 10 
 C[1][0] = 0  C[1][1] = 8  C[1][2] = 16 
 C[2][0] = 0  C[2][1] = 11  C[2][2] = 22 
==754== Profiling application: cuda/MatMult
==754== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   51.56%  4.2240us         1  4.2240us  4.2240us  4.2240us  matrix_mul(int*, int*, int*, int)
                   25.78%  2.1120us         1  2.1120us  2.1120us  2.1120us  [CUDA memcpy DtoH]
                   22.66%  1.8560us         2     928ns     672ns  1.1840us  [CUDA memcpy HtoD]
      API calls:   97.93%  103.20ms         3  34.400ms  5.3280us  103.18ms  cudaMalloc
                    1.42%  1

In [11]:
%%writefile cuda/MatMultF.cu

#include <stdio.h>
#include <math.h>
#include <time.h>
#include <sys/time.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void matrixMultiplicationKernel(float* A, float* B, float* C, int N) {

    int ROW = blockIdx.y*blockDim.y+threadIdx.y;
    int COL = blockIdx.x*blockDim.x+threadIdx.x;

    float tmpSum = 0;

    if (ROW < N && COL < N) {
        // each thread computes one element of the block sub-matrix
        for (int i = 0; i < N; i++) {
            tmpSum += A[ROW * N + i] * B[i * N + COL];
        }
    }
    C[ROW * N + COL] = tmpSum;
}

// programa principal

int main(int argc, char *argv[])
{

    if(argc!=2) {
      printf("Ha olvidado el valor de N, NxN.\n");
      exit(1);
    }
    int N = atoi(argv[1]);
    printf("tamaño= %d x %d \n", N,N);

    struct timeval fin,ini;
    long int tiempoGPU,tiempoCPU;

//    int N = 16;
    float *h_a, *h_b, *h_c;
    float *d_a, *d_b, *d_c;
    int size = N * N * sizeof(float);

    // reserva de memoria en el host
    h_a = (float *)malloc(size);
    h_b = (float *)malloc(size);
    h_c = (float *)malloc(size);

		// Inicializacion de valores en la CPU (host)
    for (int i=0; i<N; i++){
        for (int j=0; j<N; j++){
            h_a[i*N+j] = sin(i);
            h_b[i*N+j] = cos(j);
        }
    }

    // reserva de memoria en la GPU
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);

    // copia de valores a la memoria de la GPU
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);


    // dimensionar el numero de hilos ajustandolo al tamaño de la matriz
    dim3 threadsPerBlock(N, N);
    dim3 blocksPerGrid(1, 1);
        if (N*N > 1024){
            threadsPerBlock.x = 32;
            threadsPerBlock.y = 32;
            blocksPerGrid.x = ceil(double(N)/double(threadsPerBlock.x));
            blocksPerGrid.y = ceil(double(N)/double(threadsPerBlock.y));
        }
    printf("threadsPerBlock.x= %d  y threadsPerBlock.y= %d \n", threadsPerBlock.x,threadsPerBlock.y);
    printf("blocksPerGrid.x= %d  y blocksPerGrid.y= %d \n", blocksPerGrid.x,blocksPerGrid.y);

    gettimeofday(&ini,NULL);

    // Llamada al Kernel a ejecutar en la GPU
    matrixMultiplicationKernel<<<blocksPerGrid,threadsPerBlock>>>(d_a, d_b, d_c, N);

    cudaDeviceSynchronize();
    gettimeofday(&fin,NULL);

    tiempoGPU=(fin.tv_sec*1000000+fin.tv_usec)-(ini.tv_sec*1000000+ini.tv_usec);
    printf("tiempo GPU: %ld us. \n",tiempoGPU);

   // copia de resultados a la memoria de la CPU
    cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);

    float *cpu_C;
    cpu_C = (float *)malloc(size);

     gettimeofday(&ini,NULL);
    // Now do the matrix multiplication on the CPU
    float sum;
    for (int row=0; row<N; row++){
        for (int col=0; col<N; col++){
            sum = 0.f;
            for (int n=0; n<N; n++){
                sum += h_a[row*N+n]*h_b[n*N+col];
            }
            cpu_C[row*N+col] = sum;
        }
    }

    gettimeofday(&fin,NULL);
    tiempoCPU=(fin.tv_sec*1000000+fin.tv_usec)-(ini.tv_sec*1000000+ini.tv_usec);
    printf("tiempo CPU: %ld us. \n",tiempoCPU);


    double err = 0;
    // Check the result and make sure it is correct
    for (int ROW=0; ROW < N; ROW++){
        for (int COL=0; COL < N; COL++){
            err += cpu_C[ROW * N + COL] - h_c[ROW * N + COL];
        }
    }

    printf ("Error = %f ", err);printf("\n");

// liberar memoria

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);

    return 0;
}


Writing cuda/MatMultF.cu


In [12]:
!nvcc cuda/MatMultF.cu -o cuda/MatMultF
!ls -la cuda

total 1920
drwxr-xr-x 2 root root   4096 Oct 15 15:31 .
drwxr-xr-x 1 root root   4096 Oct 15 15:31 ..
-rwxr-xr-x 1 root root 971280 Oct 15 15:31 MatMult
-rw-r--r-- 1 root root   1852 Oct 15 15:31 MatMult.cu
-rwxr-xr-x 1 root root 971728 Oct 15 15:31 MatMultF
-rw-r--r-- 1 root root   3658 Oct 15 15:31 MatMultF.cu


In [13]:
!./cuda/MatMultF 32

tamaño= 32 x 32 
threadsPerBlock.x= 32  y threadsPerBlock.y= 32 
blocksPerGrid.x= 1  y blocksPerGrid.y= 1 
tiempo GPU: 73318 us. 
tiempo CPU: 171 us. 
Error = 0.000048 


Para interpretar correctamente los resultados, es importante diferenciar la ejecución de la primera vez que se utiliza la GPU al resto veces, y descartar esa primera vez por inicialización de la GPU.

## Comprobar los resultados con diferentes valores de N (tamaño de matriz NxN)

In [14]:
!./cuda/MatMultF 16

tamaño= 16 x 16 
threadsPerBlock.x= 16  y threadsPerBlock.y= 16 
blocksPerGrid.x= 1  y blocksPerGrid.y= 1 
tiempo GPU: 253 us. 
tiempo CPU: 24 us. 
Error = -0.000011 


In [15]:
!nvprof cuda/MatMultF 16

tamaño= 16 x 16 
==817== NVPROF is profiling process 817, command: cuda/MatMultF 16
threadsPerBlock.x= 16  y threadsPerBlock.y= 16 
blocksPerGrid.x= 1  y blocksPerGrid.y= 1 
tiempo GPU: 246 us. 
tiempo CPU: 20 us. 
Error = -0.000011 
==817== Profiling application: cuda/MatMultF 16
==817== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   51.73%  4.7670us         1  4.7670us  4.7670us  4.7670us  matrixMultiplicationKernel(float*, float*, float*, int)
                   26.74%  2.4640us         1  2.4640us  2.4640us  2.4640us  [CUDA memcpy DtoH]
                   21.53%  1.9840us         2     992ns     768ns  1.2160us  [CUDA memcpy HtoD]
      API calls:   99.39%  107.52ms         3  35.840ms  3.3730us  107.51ms  cudaMalloc
                    0.22%  236.44us         1  236.44us  236.44us  236.44us  cudaLaunchKernel
                    0.18%  189.76us       114  1.6640us     188ns  74.459us  cuDeviceGetAttribute
     

In [16]:
!./cuda/MatMultF 1024

tamaño= 1024 x 1024 
threadsPerBlock.x= 32  y threadsPerBlock.y= 32 
blocksPerGrid.x= 32  y blocksPerGrid.y= 32 
tiempo GPU: 6879 us. 
tiempo CPU: 13145171 us. 
Error = -0.116583 


In [17]:
!nvprof cuda/MatMultF 1024

tamaño= 1024 x 1024 
==887== NVPROF is profiling process 887, command: cuda/MatMultF 1024
threadsPerBlock.x= 32  y threadsPerBlock.y= 32 
blocksPerGrid.x= 32  y blocksPerGrid.y= 32 
tiempo GPU: 6874 us. 
tiempo CPU: 8445860 us. 
Error = -0.116583 
==887== Profiling application: cuda/MatMultF 1024
==887== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   65.71%  6.6583ms         1  6.6583ms  6.6583ms  6.6583ms  matrixMultiplicationKernel(float*, float*, float*, int)
                   18.65%  1.8898ms         1  1.8898ms  1.8898ms  1.8898ms  [CUDA memcpy DtoH]
                   15.64%  1.5846ms         2  792.32us  787.12us  797.52us  [CUDA memcpy HtoD]
      API calls:   86.01%  80.850ms         3  26.950ms  67.757us  80.705ms  cudaMalloc
                    7.10%  6.6710ms         2  3.3355ms  5.1240us  6.6658ms  cudaDeviceSynchronize
                    5.81%  5.4579ms         3  1.8193ms  979.12us  3.4574ms  cudaM

In [18]:
!./cuda/MatMultF 64

tamaño= 64 x 64 
threadsPerBlock.x= 32  y threadsPerBlock.y= 32 
blocksPerGrid.x= 2  y blocksPerGrid.y= 2 
tiempo GPU: 242 us. 
tiempo CPU: 1403 us. 
Error = 0.000739 


In [19]:
!nvprof cuda/MatMultF 64

tamaño= 64 x 64 
==939== NVPROF is profiling process 939, command: cuda/MatMultF 64
threadsPerBlock.x= 32  y threadsPerBlock.y= 32 
blocksPerGrid.x= 2  y blocksPerGrid.y= 2 
tiempo GPU: 261 us. 
tiempo CPU: 1391 us. 
Error = 0.000739 
==939== Profiling application: cuda/MatMultF 64
==939== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   65.89%  19.040us         1  19.040us  19.040us  19.040us  matrixMultiplicationKernel(float*, float*, float*, int)
                   22.04%  6.3680us         2  3.1840us  3.1680us  3.2000us  [CUDA memcpy HtoD]
                   12.07%  3.4880us         1  3.4880us  3.4880us  3.4880us  [CUDA memcpy DtoH]
      API calls:   99.23%  99.003ms         3  33.001ms  4.5410us  98.981ms  cudaMalloc
                    0.24%  242.49us       114  2.1270us     219ns  118.50us  cuDeviceGetAttribute
                    0.23%  231.49us         1  231.49us  231.49us  231.49us  cudaLaunchKernel
    

In [20]:
!./cuda/MatMultF 128

tamaño= 128 x 128 
threadsPerBlock.x= 32  y threadsPerBlock.y= 32 
blocksPerGrid.x= 4  y blocksPerGrid.y= 4 
tiempo GPU: 234 us. 
tiempo CPU: 7131 us. 
Error = 0.000598 


In [21]:
!nvprof cuda/MatMultF 128

tamaño= 128 x 128 
==957== NVPROF is profiling process 957, command: cuda/MatMultF 128
threadsPerBlock.x= 32  y threadsPerBlock.y= 32 
blocksPerGrid.x= 4  y blocksPerGrid.y= 4 
tiempo GPU: 337 us. 
tiempo CPU: 12224 us. 
Error = 0.000598 
==957== Profiling application: cuda/MatMultF 128
==957== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   57.65%  34.367us         1  34.367us  34.367us  34.367us  matrixMultiplicationKernel(float*, float*, float*, int)
                   30.81%  18.368us         2  9.1840us  8.8640us  9.5040us  [CUDA memcpy HtoD]
                   11.54%  6.8800us         1  6.8800us  6.8800us  6.8800us  [CUDA memcpy DtoH]
      API calls:   99.28%  126.69ms         3  42.231ms  3.7880us  126.68ms  cudaMalloc
                    0.23%  298.02us         1  298.02us  298.02us  298.02us  cudaLaunchKernel
                    0.17%  215.76us       114  1.8920us     197ns  82.533us  cuDeviceGetAttribute

### Ejercicio T2: Estudie el rendimiento del programa en función de N.

 Para guiarle en este estudio, se plantean a continuación diferentes cuestiones a responder:
- ¿Qué valor de N obtiene Tgpu=Tcpu? ¿Qué pasa antes y después de este N? ¿Por qué?
- ¿Cuál es el máximo valor de N que logra ejecutarse en la GPU? ¿Por qué?
- ¿Qué coste extra en las operaciones de memoria tiene la GPU frente a la CPU? ¿Qué consideraría “justo” añadir para comparar con la CPU?
